In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import ipyvolume.pylab as p3

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

Connecting cpapadop@10.28.0.34:3306


In [3]:
fetched_mesh = ta3p100.Decimation35.fetch(limit=1, as_dict=True)[0]

# Skeleton

In [ ]:
# Once I make several, small voxels it would be quite easy to turn that into a skeleton structure by just connecting voxels
# that are next to each other.

# I could use the centroids of the voxels as bone points, and also downsample the bone structure by checking to see how far
# a bone vector strays from the centroids of adjacent voxels, or rather the downstream voxels from a starting voxel.

# If I can scan across the whole mesh and start building regression lines... and correcting and branching off... hmm that might be possible.
# Basically I would start a bone at a vertex, and continue that bone as I scan through the mesh across one axis. If I find a vertex far enough
# away from my current bone then I either start a new bone OR I branch off from the current bone.
# I think that I will basically grant membership of vertices to each bone in order to create the skeleton structure. So I need to keep track
# of which vertices are assigned to each bone. Probably using a dict, and from there I can just append vertices to the bone index. This will
# also allow me to compare 2 bones (and their vertices) with each other by looking at 2 bone indices in the dict.
# I should also make a verification function to see how close the vertices are to their assigned bones.
# Depending on how I merge bones, I might get spines also defined as bones, which might be beneficial.
# Essentially I'm going through a decision tree:
# 1. If I find a vertex on its own, I create a bone.
# 2. If I find a vertex near another vertex, I keep the bone growing.
# 

class Skeleton:
    _bones = list()
    
    def __init__(self, vertices, triangles):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
    
    @property
    def vertices(self):
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def _sorted_vertices(self):
        """
        Returns sorted nonisolated vertices.
        """
        verts = self._nonisolated_vertices
        sorted_idx = verts.T[2].argsort()
        return verts[sorted_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T]) # Should I use nonisolated vertices though?
    
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def plot_verts(self, width=1024, height=1024, targeted_verts=None, **kwargs):
        if targeted_verts is None:
            verts = self.vertices
        else:
            verts = targeted_verts
        
        p3.figure(width=width, height=height)
        p3.scatter(*verts.T/1000, **kwargs)
        p3.squarelim()
        p3.show()
        
    def thick_plane(self, num_planes=50):
        z_space = np.linspace(*self.bbox[2], num=num_planes)

        starting_idx = 0
        partition_edge_idx = list([starting_idx])
        verts = self._sorted_vertices
        for j, z_edge in enumerate(z_space[1:-1]):
            for i, vert in enumerate(verts.T[2][starting_idx:]):
                if vert > z_edge:
                    starting_idx += i
                    partition_edge_idx.append(starting_idx)
                    break
        partition_edge_idx.append(-1) #(len(verts) - 1)
        
        return np.array(partition_edge_idx)
    
    @property
    def bones(self):
        return self._bones
    
    def add_bone(self, starting_point, ending_point):
        # The bones should probably be connected in some way? Though a branching bone might not want to be connected.
        self._bones.append((starting_point, ending_point))
        
    def scan(self, axis='x'):        
        if axis.lower() == 'x':
            axis_idx = 0
        elif axis.lower() == 'y':
            axis_idx = 1
        elif axis.lower() == 'z':
            axis_idx = 2
        else:
            raise ValueError("Invalid value for axis, choose between 'x', 'y', and 'z'.")
            
        verts = self._sorted_vertices
        
        bone_membership = dict()
        for vert in verts:
            

In [ ]:
skel = Skeleton()

# Voxel

In [222]:
# I should be able to allow it to Voxelize only certain parts of the mesh by changing the input BBox.

# Due to the odd shape of some of these neurons and their parts, I might really just want to use rectangular shapes that can be rotated...
# but at that point wouldn't it make more sense to create skeletons? If I figure out how to do that it would make my life (and getting
# accurate minimum distance measures) much easier. Though it would be difficult to extract the info needed for those skeletons.

# I can turn this into a skeleton by merging touching voxels, or by simple connecting the centroids of voxels by some rule (basically make a tree structure
# starting from one centroid).

# I can merge voxels by checking to see how much their adjacent faces are overlapping (and I should also ensure they don't get too big, by keeping a threshold
# either on the volume/size it can be, or by restricting it by how much empty space I'd be adding in).

class Mesh:
    def __init__(self, vertices, triangles, ignore_isolated_vertices=True):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
        self._ignore_isolated = ignore_isolated_vertices
        
        self._voxels = list()
    
    class Voxel: # Override some operators to allow direct manipulation to the bbox inside.
        """
        If I really want Voxels that are cubes, I can push the boundaries of the Mesh bounding box to fit what I need (to allow the cubes to fit edge to edge).
        """
        def __init__(self, bbox):
            self._bbox = bbox
            
        @property
        def bbox(self):
            return self._bbox
        
        @bbox.setter
        def bbox(self, bbox):
            if not isinstance(bbox, np.ndarray):
                bbox = np.array(bbox)
            
            if bbox.shape == (3, 2):
                self._bbox = bbox
            else:
                raise ValueError("Bounding box is not in required form which is: array-like with shape (3, 2).")
            
        @property
        def centroid(self):
            return self.bbox.T.mean(axis=0)
        
        def __str__(self):
            return str(self.bbox)
    
    @property
    def voxels(self):
        return self._voxels
    
    @voxels.setter
    def voxels(self, voxels):
        self._voxels = voxels
    
    @property
    def vertices(self): # Could "potentially" have the ignore_isolated_vertices_check in here... but it would mess up indices, so put in
        # another property entirely.
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T])
    
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
    
        
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def generate_voxel_lines(self):
        bboxes = np.array([voxel.bbox for voxel in self.voxels])
        return np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
    
    def plot_voxels(self, width=1024, height=1024):
        fig = p3.figure(width=width, heigh=height)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
        p3.squarelim()
        p3.show()
        
#         return fig
    
    def plot_mesh_and_voxels(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
        p3.squarelim()
        p3.show()    
    
    def merge_touching_voxels(self):
        raise NotImplementedError
        
    def debug(self, voxel_side_length=1000):
        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)

        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        bboxes = list()
        x_pairs = np.array([(x_split[i], x_split[i+1]) for i in range(len(x_split) - 1)])
        y_pairs = np.array([(y_split[i], y_split[i+1]) for i in range(len(y_split) - 1)])
        z_pairs = np.array([(z_split[i], z_split[i+1]) for i in range(len(z_split) - 1)])
        for xs in x_pairs:
            for ys in y_pairs:
                for zs in z_pairs:
                    bboxes.append((xs, ys, zs))
        bboxes = np.array(bboxes)
        
        bbox_lines = np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
        
        p3.figure(width=1024, height=1024)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles)
        [p3.plot(*lines.T/1000, color='blue') for lines in bbox_lines]
        p3.squarelim()
        p3.show()
        
        return bboxes
    
    def restrict_bboxes(self, voxel_side_length=1000): # Can improve the cubimetrics (fake word for splitting it into cubes)
        nonisolated_vertices = self._nonisolated_vertices
        
        # I should try rounding the mesh bbox up to the nearest level that is evenly divisible by the voxel_side_length
        # that way I can actually make it split into cubes nicely.
        
        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)
#         n_splits = [11] * 3
    
        x_edges, y_edges, z_edges = [np.linspace(minimum, maximum, num=axis_splits)[1:-1]
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
                
        bboxes = list()
        
        sorted_idx = nonisolated_vertices.T[0].argsort()
        x_verts = nonisolated_vertices[sorted_idx]
        splitter = x_verts.T[0].searchsorted(x_edges)
        x_split = np.split(x_verts, splitter)
        for x_block in x_split:
            if len(x_block) > 0:
                sorted_idx = x_block.T[1].argsort()
                y_verts = x_block[sorted_idx]
                splitter = y_verts.T[1].searchsorted(y_edges)
                y_split = np.split(y_verts, splitter)
                if len(y_split) > 0:
                    for y_block in y_split:
                        if len(y_block) > 0:
                            sorted_idx = y_block.T[2].argsort()
                            z_verts = y_block[sorted_idx]
                            splitter = z_verts.T[2].searchsorted(z_edges)
                            z_split = np.split(z_verts, splitter)
                            if len(z_split) > 0:
                                for z_block in z_split:
                                    if len(z_block) > 0:
                                        bboxes.append(self.get_bbox(z_block))
            
        return bboxes
    
    def initialize_voxels(self, voxel_side_length=1000):
        bboxes = self.restrict_bboxes(voxel_side_length=voxel_side_length)
        self._voxels = [self.Voxel(bbox) for bbox in bboxes]
        return self.voxels

In [234]:
mesh = Mesh(fetched_mesh['vertices'], fetched_mesh['triangles'])
start = time.time()
mesh.initialize_voxels(5000)
time.time() - start, len(mesh.voxels)

(0.22087335586547852, 751)

In [230]:
mesh.plot_mesh_and_voxels()